# Example of Importing File with WxPython File Dialog

### Workflow for importing single function
1. Call Function read_file to initiate file dialog
2. Launch File Dialog and get file name
3. Get the File Extension
4. Depending on the file extension call proper pandas import function
    a. CSV file should call pd.read_csv
    b. Excel file (xlsx or xls) should call pd.read_excel
    c. All other files try calling read_table if it works
    
### This could be improved by possibly opening the file and reading a line determining the delimiter and whether there is a header row and then calling read_table or read_csv with the proper parameters

In [ ]:
#Code for opening file dialog and importing single table into pandas
import wx
import pandas as pd

def read_file(fname='',wildcard=''): #read em file from csv
    """Read EM DATA CSV file
    Keyword arguments:    
    fname -- file name if you know it already or '' to launch file dialog
    wildcard -- pass wildcard if you want to filter for certain file type
    returns file extensions,filename, and filepath 
    """
    if fname=='':
        fparam=get_file_name(wildcard)
        #print fparam[1]
        fpath=fparam[0]
        fname=fparam[1]
        if fname==None:
            ext=None
        else:
            dot_index=fname.find('.')
            dist_from_end=len(fname)-dot_index-1
            ext=fname[-dist_from_end:]
    return (ext, fname, fpath)
    

def get_file_name(wildcard): #generate file dialog to pick file
    """Launches file dialog
    Keyword arguments:    
    wildcard -- file type filter
    
    returns filename and path 
    """
    app = wx.App(None)
    style = wx.FD_OPEN | wx.FD_FILE_MUST_EXIST #this routine is for one file at a time
    dialog = wx.FileDialog(None, 'Open', wildcard=wildcard, style=style)
    if dialog.ShowModal() == wx.ID_OK:
        path = dialog.GetPath()
        fname = dialog.GetFilename()
    else:
        path = None
        fname = None
    dialog.Destroy()
    return (path, fname)

def showmsg(msg, msgtype, title):
    """Generates a message box with WX
    Keyword arguments:    
    msg -- msg to put in box
    msgtype -- type of message box
    title -- title of the message box
    
    returns a message box with title and msg
    """
    app = wx.App(redirect=False)
    
    if msgtype == 'ERROR':
        wx.MessageBox(msg, title, style=wx.OK|wx.ICON_ERROR)
    elif msgtype == 'Exclamation':
        wx.MessageBox(msg, title, style=wx.OK|wx.ICON_EXCLAMATION)
    else:
        wx.MessageBox('Unknown Error Occured', 'Error', style=wx.OK|wx.ICON_ERROR)
    return

# Step 4

In [ ]:
#Now the import code
fparams=read_file()
print fparams[0]
if fparams[2] == None:
    showmsg('No File was Selected For Processing \n\rRerun this script and select a file', 'ERROR', 'Error No File Selected')
elif fparams[0]=='csv':
    df=pd.read_csv(fparams[2])
    print df.head()
elif fparams[0] == 'xlsx' or fparams[0] == 'xls':
    df=pd.read_excel(fparams[2])
    print df.head()
else:    
    try:
        df=pd.read_table(fparams[2])
    except:
        showmsg('Pandas is not able to import the file', 'ERROR', 'Not Importable File')


# Multiple File Imports


In [ ]:
def get_multiple_file_names(wildcard=''): #generate file dialog to pick file
    """Launches file dialog
    Keyword arguments:    
    wildcard -- file type filter
    
    returns filenames and paths
    """
    app = wx.App(None)
    style = wx.FD_OPEN | wx.FD_FILE_MUST_EXIST | wx.FD_MULTIPLE #this last tag is used for selecting multiple files
    dialog = wx.FileDialog(None, 'Open', wildcard=wildcard, style=style)
    if dialog.ShowModal() == wx.ID_OK:
        paths = dialog.GetPaths()
        fnames = dialog.GetFilenames()
    else:
        paths = None
        fnames = None
    dialog.Destroy()
    return (paths, fnames)

def extract_ext(fname=''):
    """Extracts extensions from filenames
    Keyword arguments:    
    fname -- filename    
    returns extension string
    """
    if fname == '':
        return ''
    else:
        dot_index=fname.find('.')
        dist_from_end=len(fname)-dot_index-1
        ext=fname[-dist_from_end:]
        return ext
    
def import_file(path,ext):
    """Checks file extension and chooses correct pandas read function good for xls xlsx csv and any text table
    could be extended to shapefiles with geopandas or grd files with steno_3d_surfer.
    Keyword arguments:    
    path -- file type filter
    ext -- file extension
    
    returns pandas data frame
    """
    if path == None:
        showmsg('No File was Selected For Processing \n\rRerun this script and select a file', 'ERROR', 'Error No File Selected')
    elif ext=='csv':
        df=pd.read_csv(path)
        print df.head()
    elif ext == 'xlsx' or ext == 'xls':
        df=pd.read_excel(path)
        print df.head()
    else:    
        try:
            df=pd.read_table(path)
            print df.head()
        except:
            showmsg('Not a Table or Text File', 'ERROR', 'File cannot be imported this way')

#execution code            
fparam=get_multiple_file_names()
exts=[extract_ext(item) for item in fparam[1]] #list comprehension to extract all extensions
df_list=[import_file(fparam[0][i],exts[i]) for i in range(len(exts))] #list comprehension to read files and generate dataframe
df_dict=zip(fparam[1],df_list) # creates dictionary of data frames with filename as key with zip